In [7]:
import numpy as np
import pandas as pd
from pyspark.sql import functions as f

In [15]:
# path = '/Users/diaz ivan/airflow/etl/sources/I94MODE.csv'
# dim_arrival_mode = pd.read_csv('I94MODE.csv', sep=',')
dim_arrival_mode = spark.read.csv('I94MODE.csv', sep=',', header=True, inferSchema=True)
dim_arrival_mode.toPandas().head()

,ID,Mode
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


In [16]:
#dim_arrival_mode = dim_arrival_mode.rename(columns={'ID':"code", 'Mode':'name'})
dim_arrival_mode = dim_arrival_mode.toDF('code', 'mode')
dim_arrival_mode.columns

['code', 'mode']

In [21]:
dim_arrival_mode.toPandas()

AttributeError: 'DataFrame' object has no attribute 'toPandas'

In [20]:
dim_arrival_mode = dim_arrival_mode.withColumn('mode', f.lower(f.col('mode'))).toPandas()

AttributeError: 'DataFrame' object has no attribute 'withColumn'